In [1]:
from datasets import load_dataset,Dataset
dataset = load_dataset("squad_kor_v1")

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files #0: 100%|██████████| 1/1 [00:00<00:00, 579.72obj/s]

Extracting data files #1: 100%|██████████| 1/1 [00:00<00:00, 616.54obj/s]


Dataset squad_kor_v1 downloaded and prepared to /opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


100%|██████████| 2/2 [00:00<00:00, 411.45it/s]


In [2]:
from datasets import concatenate_datasets,Dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [3]:
import random

In [30]:
dataset['train']['context'][3]

'1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.'

In [11]:
import pandas as pd
tmp = pd.DataFrame(dataset['train'])

In [14]:
tmp

,id,title,context,question,answers
0,6566495-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,"{'text': ['교향곡'], 'answer_start': [54]}"
1,6566495-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,"{'text': ['1악장'], 'answer_start': [421]}"
2,6566495-0-2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,"{'text': ['베토벤의 교향곡 9번'], 'answer_start': [194]}"
3,6566518-0-0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,"{'text': ['파우스트'], 'answer_start': [15]}"
4,6566518-0-1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,"{'text': ['합창교향곡'], 'answer_start': [354]}"
...,...,...,...,...,...
60402,6467478-1-1,뉴델리_메탈로-베타락타마제,"유전자의 이름은 인도의 수도 뉴델리의 이름을 따 붙여졌는데, 이는 2009년 용 (...",뉴델리 메탈로 베타락마제가 처음 감염 된 지역은 어디인가?,"{'text': ['인도'], 'answer_start': [73]}"
60403,6467478-2-0,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",균은 유전자를 균에게 전달 할 수있는데 이러한 현상을 나타낸 용어는 무엇인가?,"{'text': ['유전자 전달'], 'answer_start': [253]}"
60404,6467478-2-1,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",박테리아가 NDM-1 유전자를 가지고 있을때 발생하는 전파를 분석하기위해 사용된 영...,"{'text': ['37건'], 'answer_start': [129]}"
60405,6490801-2-0,뉴델리_메탈로-베타락타마제,"2010년 8월, 저널 The Lancet Infectious Diseases에 최...",NDM-1 유전자를 가진 박테리아가 감수성을 보인 폴리믹슨 계열 항생제는?,"{'text': ['콜리스틴'], 'answer_start': [404]}"


In [9]:
type(tmp['answers'][0])

dict

In [19]:
a = Dataset.from_pandas(tmp)

In [11]:
from datasets import load_from_disk
df = load_from_disk('/opt/ml/input/data/train_dataset/')
df = pd.DataFrame(df['train'])
type(df['answers'][0])

dict

In [18]:
b = Dataset.from_pandas(df)

In [20]:
concatenate_datasets([a,b])

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 64359
})

In [25]:
from datasets import DatasetDict
def aug_data(data, new_data):
    assert type(new_data) == DatasetDict, f'it mustnbe a Dataset type but you give {type(new_data)}'
    train_df = Dataset.from_pandas(pd.DataFrame(new_data['train']))
    validation_df = Dataset.from_pandas(pd.DataFrame(new_data['validation']))
    train_data = concatenate_datasets([data['train'],train_df])
    validation_data = concatenate_datasets([data['validation'],validation_df])
    aug_dataset = DatasetDict({'train': train_data, 'validation' : validation_data})
    return aug_dataset

In [1]:
df = load_from_disk('/opt/ml/input/data/train_dataset/')
dataset = load_dataset("squad_kor_v1")

NameError: name 'load_from_disk' is not defined

In [27]:
concat_df = aug_data(df, dataset)

In [28]:
concat_df

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 64359
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 6014
    })
})

In [64]:
dataset['train'].features['answers'] # korquad

Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)

In [55]:
df['train'].features['answers']   # competiotion MRC

{'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [63]:
dataset['train']

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 60407
})

In [62]:
from datasets import Sequence
def answer_change(example):
    example['answers'] = {'answer_start' : Sequence(feature = example.features['answers'].feature['answer_start']),
                        'text' : Sequence(feature = example.features['answers'].feature['text'])
                            }
    return example
dataset['train'].map(answer_change)

  0%|          | 0/60407 [00:00<?, ?ex/s]

KeyError: 'answers'

In [39]:
df['train'].features['answers']

{'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [35]:
df =df['train'].remove_columns(['document_id', '__index_level_0__'])

In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})

In [59]:
a = list(df['train'].remove_columns(['document_id', '__index_level_0__']).features)

In [62]:
dataset['train'].features([a])

TypeError: 'Features' object is not callable